<h1 style="color:#65AE11;">非默认流中的内存复制</h1>

本节中，你将在非默认流中执行主机和设备间的内存传输。

<h2 style="color:#65AE11;">目标</h2>

完成本节内容的学习后，你将：

* 了解如何创建固定内存（可在非默认流中进行异步传输）
* 能够在非默认流中执行主机到设备的内存传输
* 能够在非默认流中执行设备到主机的内存传输
* 能够执行流同步

<h2 style="color:#65AE11;">演示</h2>

运行以下单元以加载本节的视频演示。看完演示后，继续完成此 Notebook 的余下部分。

In [ ]:
%%html
<video width="800" controls>
  <source src="https://dli-lms.s3.us-east-1.amazonaws.com/assets/s-ac-01-v1-zh/07-memcpy-streams-01.mp4" type="video/mp4">
  Your browser does not support HTML video.
</video>

<h2 style="color:#65AE11;">分配固定内存</h2>

要异步复制数据，CUDA 需对其位置作出假设。典型的主机内存会使用[分页](https://en.wikipedia.org/wiki/Paging)，这样除了 RAM 之外，数据还可存储在某个备份存储设备上（如物理磁盘）。

固定（或锁页）内存会绕过主机操作系统分页，在 RAM 中存储所分配的内存。在非默认流中异步传输内存时，必须使用锁页（或固定）内存。

固定内存会阻止将数据存储在某些备份存储设备上，因此是一个受限资源，请务必当心，不要过度使用。

固定主机内存通过 `cudaMallocHost` 进行分配：

```c
const uint64_t num_entries = 1UL << 26;
uint64_t *data_cpu;
cudaMallocHost(&data_cpu, sizeof(uint64_t)*num_entries);
```

<h2 style="color:#65AE11;">非默认流中主机到设备的内存传输</h2>

通过使用类似于 `cudaMemcpy` 的 `cudaMemcpyAsync`，你可在非默认流中将固定主机内存传输到 GPU 显存，但需提供第 5 个流标识符参数：

```c
cudaStream_t stream;
cudaStreamCreate(&stream);

const uint64_t num_entries = 1UL << 26;

uint64_t *data_cpu, *data_gpu;

cudaMallocHost(&data_cpu, sizeof(uint64_t)*num_entries);
cudaMalloc(&data_gpu, sizeof(uint64_t)*num_entries);

cudaMemcpyAsync(data_gpu, 
                data_cpu, 
                sizeof(uint64_t)*num_entries, 
                cudaMemcpyHostToDevice, 
                stream);
```

<h2 style="color:#65AE11;">非默认流中设备到主机的内存传输</h2>

通过使用 `cudaMemcpyAsync`，你也可在非默认流中将 GPU 显存传输到固定主机内存：

```c
// Assume data is already present on the GPU, and that `data_cpu` is pinned.

cudaMemcpyAsync(data_cpu, 
                data_gpu, 
                sizeof(uint64_t)*num_entries, 
                cudaMemcpyDeviceToHost, 
                stream);
```

与所有现代 GPU 一样，具有 2 个或更多复制引擎的 GPU 设备可以同时在不同的非默认流中执行主机到设备和设备到主机的内存传输。稍后，你需在本课程中亲自执行此操作。

<h2 style="color:#65AE11;">流同步</h2>

使用 `cudaStreamSynchronize` 将导致主机代码阻塞，直到给定的流完成其操作。如果需要保证流工作完成，应使用流同步，例如，在主机代码需要等待非默认流中的异步内存传输完成时：

```c
// Assume data is already present on the GPU, and that `data_cpu` is pinned.

cudaMemcpyAsync(data_cpu, 
                data_gpu, 
                sizeof(uint64_t)*num_entries, 
                cudaMemcpyDeviceToHost, 
                stream);

// Block until work (in this case memory transfer to host) in `stream` is complete.
cudaStreamSyncronize(stream);

// `data_cpu` transfer to host via `stream` is now guaranteed to be complete.
checkResultCpu(data_cpu);
```

<h2 style="color:#65AE11;">练习：在非默认流中执行内存传输</h2>

打开并重构 [*07_Memcpy_in_Streams/baseline_cipher/baseline.cu*](baseline_cipher/baseline.cu)，以在非默认流中执行主机到设备和设备到主机的内存传输。

使用 JupyterLab 终端并从 *07_Memcpy_in_Streams/baseline_cipher* 目录内运行 `make profile`，为重构后的应用生成报告文件。（详情请参阅 [*Makefile*](baseline_cipher/Makefile)）。

在 Nsight Systems 中打开报告文件。如果你已关闭 Nsight Systems 选项卡，可以按照 [*Nsight Systems 设置*](../04_Nsight_Systems_Setup/Nsight_Systems_Setup.ipynb)中的说明重新打开。在这里提醒各位，密码为 `nvidia`。

如果操作成功，你应能注意到 Nsight Systems 可视化的时间表中显示内存传输现在发生在非默认流中，如下方屏幕截图所示。

如你遇到困难，请参阅 [07_Memcpy_in_Streams/baseline_cipher/baseline_solution.cu](../07_Memcpy_in_Streams/baseline_cipher/baseline_solution.cu)。

![memcpy_in_stream](images/memcpy_in_stream.png)

<h2 style="color:#65AE11;">知识检查</h2>

请回答下列问题，确认你已了解本节的主要目标。你可以通过单击问题下方的`...`单元来显示每个问题的答案。

---

**现在你已知道如何处理默认流行为，请说明在上述练习（如上文屏幕截图所示）中，为何我们未看到内存传输与核函数执行之间或者主机到设备与设备到主机的内存传输之间存在任何重叠（即使是在非默认流中执行内存传输也是如此）。**

**答案：**

发布到默认流中的命令会待所有其他非默认流命令完成之后再开始运行，而且将会阻止任何其他非默认流执行操作，直至其运行完毕为止。

在上述练习中，由于我们未指定非默认流，因此核函数启动 `decrypt_gpu` 发生在默认流中。此命令待非默认流中主机到设备的内存传输完成之后才开始运行，且会阻止设备到主机的内存传输，直至其运行完毕为止。

---

**通过 `malloc` 在主机上分配的内存能否使用 `cudaMemcpyAsync` 进行异步传输？**

1. 是
2. 否

**答案：2。**

主机操作系统内存分页无法保证使用立即寻址方式在 RAM 中找到任何内存的位置，而是可能会通过分页将内存存储在 RAM 之外。

要在非默认流中将内存异步传输到主机或从主机传输内存，内存必须为锁页或固定内存。为此，我们使用 `cudaMallocHost` 而非 `malloc`。

---

**使用非默认流中传输数据的主机代码将等待内存传输完成，然后再开始工作。**

1. True
2. 错误

**答案：2。**

必须使用 `cudaStreamSynchronize` 来阻止主机代码继续运行，直到给定流中的工作完成。

---

<h2 style="color:#65AE11;">下一步</h2>

虽然您已学习如何在非默认流中执行核函数启动和内存传输，但上两节课并未使密码应用程序真正获得性能提升。

在随后的几节课中，您将学习如何执行数据复制与计算的重叠，并将开始看到使用并发流所带来的实际性能提升。

请继续学习下一节：[*数据复制与计算重叠的注意事项*](../08_Copy_Compute_Considerations/Copy_Compute_Considerations.ipynb)。

<h2 style="color:#65AE11;">选择进一步学习</h2>

以下内容供有时间和兴趣的学生对与本课程相关的主题进行更深入的学习。

* 某些memcpy操作的`async`后缀“是一个错误的名称，因为每个函数根据传递给该函数的参数可能表现出同步或异步行为”。有关更多详细信息，请参见[CUDA Runtime文档](https://docs.nvidia.com/cuda/cuda-runtime-api/api-sync-behavior.html#api-sync-behavior)。

* 对于使用统一内存的应用程序的那些人，请参阅[Stack Overflow提供的答案](https://stackoverflow.com/questions/23518299/unified-memory-and-streams-in-c)（更多的细节，请参考[这个文档](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#um-coherency-hd))。